In [1]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go

def get_stock_data(ticker, period='1y'):
    try:
        yahoo_ticker = ticker + '.NS'
        stock = yf.Ticker(yahoo_ticker)
        data = stock.history(period=period)
        info = stock.info
        sector = info.get('industry', 'N/A')
        stock_data = {'Ticker': ticker, 'Data': data, 'Sector': sector}
        return stock_data
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

def fetch_stock_data(stock_symbols, period='1y'):
    df_list = []

    for stock_symbol in stock_symbols:
        stock_data = get_stock_data(stock_symbol, period)

        if stock_data:
            stock_data['Data']['Stock'] = stock_symbol
            df_list.append(stock_data['Data'])

    df = pd.concat(df_list)
    return df

def calculate_correlation(df):
    correlation_matrix = df.pivot_table(index='Date', columns='Stock', values='Close').corr()
    return correlation_matrix

def analyze_stock(stock_data):
    selected_column = 'Close'
    stock_ticker = stock_data['Stock'].iloc[0]

    print("\nStock Name:", stock_ticker)
    std_deviation = stock_data[selected_column].std()
    mean_stock_price = stock_data[selected_column].mean()
    current_stock_price = stock_data[selected_column].iloc[-1]
    std_range = (current_stock_price - mean_stock_price) / std_deviation
    z_score = (current_stock_price - mean_stock_price) / std_deviation

    print("Standard deviation in the last one year:", std_deviation)
    print("Mean Stock Price:", mean_stock_price)
    print("Current Stock Price:", current_stock_price)
    print("Z-Score value:", z_score)

    if abs(std_range) < 1:
        print("Current stock price is within 1 standard deviation from the mean.")
    elif abs(std_range) < 2:
        print("Current stock price is within 2 standard deviations from the mean.")
    elif abs(std_range) < 3:
        print("Current stock price is within 3 standard deviations from the mean.")
    else:
        print("Current stock price is more than 3 standard deviations from the mean.")

def find_high_correlation_pairs(correlation_matrix, threshold=0.90):
    high_corr_pairs = []

    for stock1 in correlation_matrix.columns:
        for stock2 in correlation_matrix.columns:
            if stock1 < stock2 and abs(correlation_matrix.loc[stock1, stock2]) > threshold:
                high_corr_pairs.append((stock1, stock2, correlation_matrix.loc[stock1, stock2]))

    return pd.DataFrame(high_corr_pairs, columns=['Stock1', 'Stock2', 'Correlation']).sort_values(by='Correlation', ascending=False)

def calculate_price_ratio_stats(wide_df, stock1, stock2):
    if stock1 not in wide_df.columns or stock2 not in wide_df.columns:
        print(f"Error: {stock1} or {stock2} not found in DataFrame columns.")
        return None

    LogicValue = wide_df[stock1].tail(1) > wide_df[stock2].tail(1)

    if LogicValue[0] == True:
        wide_df['PRatio'] = wide_df[stock1] / wide_df[stock2]
    else:
        wide_df['PRatio'] = wide_df[stock2] / wide_df[stock1]

    wide_df['PRatio_Mean'] = wide_df['PRatio'].mean()
    wide_df['Correlation'] = wide_df[stock1].corr(wide_df[stock2])
    wide_df['Z_Score' + stock1] = ((wide_df[stock1] - wide_df[stock1].mean()) / (wide_df[stock1].std()))
    wide_df['Z_Score' + stock2] = ((wide_df[stock2] - wide_df[stock2].mean()) / (wide_df[stock2].std()))

    return wide_df.tail(2)

def plot_stock_price_analysis(wide_df2):
    fig = go.Figure()

    for column in wide_df2.columns:
        fig.add_trace(go.Scatter(x=wide_df2.index, y=wide_df2[column], mode='lines', name=column))

    fig.update_layout(
        title='Stocks Price Analysis',
        xaxis=dict(title='Date'),
        yaxis=dict(title='Values'),
        showlegend=True,
    )

    fig.show()

def main():
    print("Welcome to Stock Analysis Tool!")

    while True:
        # User inputs
        stock_symbols = input("Enter a list of stock symbols separated by commas: ").split(',')
        analysis_period = input("Enter the period for stock data analysis (e.g., '1y' for 1 year): ")
        correlation_threshold = float(input("Enter the correlation threshold for displaying top pairs (e.g., 0.90): "))

        # Fetch stock data
        df = fetch_stock_data(stock_symbols, period=analysis_period)

        # Calculate correlation
        correlation_matrix = calculate_correlation(df)

        # Find pairs with high correlation
        df_high_corr_pairs = find_high_correlation_pairs(correlation_matrix, threshold=correlation_threshold)

        # Display high correlation pairs
        print("\nPairs with High Correlation:")
        print(df_high_corr_pairs)

        if not df_high_corr_pairs.empty:
            # Select a pair for detailed analysis
            selected_pair_index = int(input("Enter the index of the pair you want to analyze (e.g., 0 for the top pair): "))
            selected_pair = df_high_corr_pairs.iloc[selected_pair_index]
            stock_1 = selected_pair['Stock1']
            stock_2 = selected_pair['Stock2']
            
            # Get statistical analysis of the selected pair
            calculate_price_ratio_stats(df[df['Stock'].isin([stock_1, stock_2])], stock_1, stock_2)

            # Analyze each stock in the selected pair
            for stock_symbol in [stock_1, stock_2]:
                stock_data = df[df['Stock'] == stock_symbol]
                analyze_stock(stock_data)

            # Data preprocessing and feature creation
            pricedata = df[df['Stock'].isin([stock_1, stock_2])][['Close', 'Stock']]
            wide_df = pricedata.pivot(columns='Stock', values='Close')

            LogicValue = wide_df[stock_1].tail(1) > wide_df[stock_2].tail(1)
            if LogicValue[0] == True:
                wide_df['PRatio'] = wide_df[stock_1] / wide_df[stock_2]
                wide_df['PRatio_Mean'] = wide_df['PRatio'].mean()
                wide_df['Correlation'] = wide_df[stock_1].corr(wide_df[stock_2])
                wide_df['Z_Score' + stock_1] = ((wide_df[stock_1] - wide_df[stock_1].mean()) / (wide_df[stock_1].std()))
                wide_df['Z_Score' + stock_2] = ((wide_df[stock_2] - wide_df[stock_2].mean()) / (wide_df[stock_2].std()))
            else:
                wide_df['PRatio'] = wide_df[stock_2] / wide_df[stock_1]
                wide_df['PRatio_Mean'] = wide_df['PRatio'].mean()
                wide_df['Correlation'] = wide_df[stock_2].corr(wide_df[stock_1])
                wide_df['Z_Score' + stock_1] = ((wide_df[stock_1] - wide_df[stock_1].mean()) / (wide_df[stock_1].std()))
                wide_df['Z_Score' + stock_2] = ((wide_df[stock_2] - wide_df[stock_2].mean()) / (wide_df[stock_2].std()))

            # Plot stock price analysis
            plot_stock_price_analysis(wide_df)


        # Ask user if they want to continue
        another_analysis = input("Do you want to perform another analysis? (yes/no): ").lower()
        if another_analysis != 'yes':
            break

if __name__ == "__main__":
    main()

Welcome to Stock Analysis Tool!
Enter a list of stock symbols separated by commas: SBIN,ICICIBANK,HDFCBANK,AXISBANK,KOTAKBANK,PNB,BANKBARODA,YESBANK,IDBI,BANKINDIA,CENTRALBK,UNIONBANK,CANBK,RBLBANK,INDUSINDBK,IDFCFIRSTB,FEDERALBNK
Enter the period for stock data analysis (e.g., '1y' for 1 year): 1
Enter the correlation threshold for displaying top pairs (e.g., 0.90): .95

Pairs with High Correlation:
Empty DataFrame
Columns: [Stock1, Stock2, Correlation]
Index: []
Do you want to perform another analysis? (yes/no): yes
Enter a list of stock symbols separated by commas: SBIN,ICICIBANK,HDFCBANK,AXISBANK,KOTAKBANK,PNB,BANKBARODA,YESBANK,IDBI,BANKINDIA,CENTRALBK,UNIONBANK,CANBK,RBLBANK,INDUSINDBK,IDFCFIRSTB,FEDERALBNK
Enter the period for stock data analysis (e.g., '1y' for 1 year): 1
Enter the correlation threshold for displaying top pairs (e.g., 0.90): .80

Pairs with High Correlation:
Empty DataFrame
Columns: [Stock1, Stock2, Correlation]
Index: []
Do you want to perform another analysis

In [ ]:
#SBIN,ICICIBANK,HDFCBANK,AXISBANK,KOTAKBANK,PNB,BANKBARODA,YESBANK,IDBI,BANKINDIA,CENTRALBK,UNIONBANK,CANBK,RBLBANK,INDUSINDBK,IDFCFIRSTB,FEDERALBNK